# Data Loader

In [1]:
from pathlib import Path
from typing import List, Dict, Any
from langchain_community.document_loaders import PyPDFLoader,PyMuPDFLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders.excel import UnstructuredExcelLoader
import numpy as np 

d:\GenAIPractise\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = Path("data")
pdf_files = list(data_path.rglob("*.pdf"))
loader = PyPDFLoader(str(pdf_files[0]))
documents = loader.load()
print(f"Loaded {len(documents)} documents from PDF.")

Loaded 3 documents from PDF.


In [3]:
# Load PDF Files 
def load_pdfs_from_directory(data_dir: str) -> List[Any]:
    """ Load all pdf files from the data directory
        Supported formats: .pdf
    """
    data_path = Path(data_dir)
    pdf_files = list(data_path.rglob("*.pdf"))
    all_documents = []
    for pdf_file in pdf_files:
        print(f"Loading PDF file: {pdf_file}")
        loader = PyMuPDFLoader(str(pdf_file))
        documents = loader.load()
        all_documents.extend(documents)
        print(f"Loaded {len(documents)} documents from {pdf_file}")
    return all_documents


In [4]:
docs = load_pdfs_from_directory(data_dir="data")

Loading PDF file: data\AgenticRAG.pdf
Loaded 3 documents from data\AgenticRAG.pdf
Loading PDF file: data\Linear Regression.pdf
Loaded 2 documents from data\Linear Regression.pdf


In [5]:
print(docs[0].page_content)

LLMs enable the development of intelligent agents capable of tackling 
complex, non-repetitive tasks that defy description as deterministic 
workflows. By spli_ing reasoning into steps in different ways and 
orchestrating them in a relatively simple way, agents can demonstrate a 
significantly higher task completion rate on complex open tasks. 
This agent-based approach can be applied across numerous domains, 
including RAG systems, which we discussed in Chapter 4. As a reminder, 
what exactly is agentic RAG? Remember, a classic pa_ern for a RAG system is 
to retrieve chunks given the query, combine them into the context, and ask an 
LLM to generate an answer given a system prompt, combined context, and the 
question. 
We can improve each of these steps using the principles discussed above 
(decomposition, tool calling, and adaptation): 
Dynamic retrieval hands over the retrieval query generation to the LLM. 
It can decide itself whether to use sparse embeddings, hybrid methods, 
keywo

In [6]:
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100,separators=["\n\n","\n"," ",""])
chunks=splitter.split_documents(docs)

In [8]:
texts=[chunk.page_content for chunk in chunks]
print(texts)
# embeddings=SentenceTransformer(model_name_or_path=="all-MiniLM-L6-v2")

['LLMs enable the development of intelligent agents capable of tackling \ncomplex, non-repetitive tasks that defy description as deterministic \nworkflows. By spli_ing reasoning into steps in different ways and \norchestrating them in a relatively simple way, agents can demonstrate a \nsignificantly higher task completion rate on complex open tasks. \nThis agent-based approach can be applied across numerous domains, \nincluding RAG systems, which we discussed in Chapter 4. As a reminder, \nwhat exactly is agentic RAG? Remember, a classic pa_ern for a RAG system is \nto retrieve chunks given the query, combine them into the context, and ask an \nLLM to generate an answer given a system prompt, combined context, and the \nquestion. \nWe can improve each of these steps using the principles discussed above \n(decomposition, tool calling, and adaptation): \nDynamic retrieval hands over the retrieval query generation to the LLM. \nIt can decide itself whether to use sparse embeddings, hybrid

In [9]:
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings=model.encode(texts,show_progress_bar=True)

Batches: 100%|██████████| 1/1 [00:04<00:00,  4.06s/it]


In [10]:
# embeddings.shape[1]
emb1=np.array(embeddings.astype("float32"))
emb1

array([[-0.01537459, -0.00807518, -0.01349245, ...,  0.05261908,
        -0.02061925,  0.05256796],
       [ 0.01792927,  0.0146074 ,  0.01082158, ...,  0.01337339,
        -0.07734995,  0.03242376],
       [ 0.0203466 ,  0.01087702, -0.00036829, ..., -0.01249124,
        -0.087068  ,  0.01081242],
       ...,
       [-0.05097468,  0.0071922 , -0.06002007, ..., -0.00390866,
         0.04024328, -0.06602587],
       [ 0.08606234, -0.06391905,  0.01570605, ...,  0.0404456 ,
         0.07214393, -0.04788442],
       [-0.04269894,  0.00727912, -0.01060689, ...,  0.01063519,
         0.07772786, -0.02818427]], shape=(11, 384), dtype=float32)

In [11]:
import os
import faiss
import numpy as np
import pickle


In [12]:
metadata=[{"text":chunk.page_content} for chunk in chunks]
metadata

[{'text': 'LLMs enable the development of intelligent agents capable of tackling \ncomplex, non-repetitive tasks that defy description as deterministic \nworkflows. By spli_ing reasoning into steps in different ways and \norchestrating them in a relatively simple way, agents can demonstrate a \nsignificantly higher task completion rate on complex open tasks. \nThis agent-based approach can be applied across numerous domains, \nincluding RAG systems, which we discussed in Chapter 4. As a reminder, \nwhat exactly is agentic RAG? Remember, a classic pa_ern for a RAG system is \nto retrieve chunks given the query, combine them into the context, and ask an \nLLM to generate an answer given a system prompt, combined context, and the \nquestion. \nWe can improve each of these steps using the principles discussed above \n(decomposition, tool calling, and adaptation): \nDynamic retrieval hands over the retrieval query generation to the LLM. \nIt can decide itself whether to use sparse embedding

In [13]:
# Creating index for FAISS vector store using L2 distance metric same embeddings shape

index=faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings.astype("float32")))


In [14]:
# Store the index and metadata to disk

os.makedirs("faiss_index",exist_ok=True)
faiss_path="faiss_index/faiss.index"
faiss.write_index(index,faiss_path)
metadata_path="faiss_index/metadata.pkl"
with open (metadata_path,"wb") as f:
    pickle.dump(metadata,f)
print("FAISS index and metadata saved successfully.")


FAISS index and metadata saved successfully.


In [15]:
load_index = faiss.read_index(faiss_path)
with open(metadata_path,"rb") as f:
    loaded_metadata=pickle.load(f)

In [16]:
queryemb = model.encode(["What is Linear Regression?"]).astype("float32")
D, I = load_index.search(np.array(queryemb),k=3)
print(D[0],I[0])

[0.52223206 0.8408511  0.9833946 ] [ 8 10  9]


In [ ]:
print(D,I)

In [60]:
query = "What is Multi Agent Architecture?"
queryemb = model.encode([query]).astype("float32")
D, I = load_index.search(np.array(queryemb),k=3)
results=[]
for idx,distance in zip(I[0],D[0]):
    meta = loaded_metadata[idx]
    result={"document":meta["text"],"distance":distance,"index":idx}
    # print(result)
    results.append(result)
# print(results)



In [18]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq

In [53]:
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
chat_groq_llm = ChatGroq(groq_api_key=groq_api_key, model="llama-3.1-8b-instant", temperature=0)

In [50]:
texts=[]
texts.append([results[i]['document'] for i in range(len(results))])

In [51]:
texts

[['Linear Regression (No Code Guide) \n1. Introduction to Linear Regression \n• \nDefinition: Linear regression is a statistical method used to model the relationship between a \ndependent variable and one or more independent variables. \n• \nPurpose: It helps in predicting outcomes and understanding how variables are related. \n• \nApplications: Business forecasting, risk analysis, economics, medical research, and machine \nlearning. \n \n2. Core Concepts \n• \nDependent Variable (Y): The outcome you want to predict. \n• \nIndependent Variable (X): The input or predictor. \n• \nLine of Best Fit: A straight line that minimizes the difference between predicted and actual \nvalues. \n• \nEquation: ( y = mx + c )  \no \n(m): slope (effect of X on Y) \no \n(c): intercept (value of Y when X = 0) \n \n3. Assumptions of Linear Regression \n• \nLinearity: Relationship between X and Y is linear. \n• \nIndependence: Observations are independent of each other. \n• \nHomoscedasticity: Constant var

In [61]:
prompt = f""" Summarize the following for the query:'{query}'\n\n Context:\n{texts}\n\nSummary:"""
response = chat_groq_llm.invoke(prompt)
print(response.content)

The provided text does not mention "Multi Agent Architecture." It appears to be a guide on Linear Regression, a statistical method used to model the relationship between a dependent variable and one or more independent variables. 

If you are looking for information on Multi Agent Architecture, I can provide a general summary. Multi-Agent Architecture is a design pattern used in artificial intelligence and multi-agent systems. It involves designing a system that consists of multiple autonomous agents that interact with each other and their environment to achieve a common goal. The architecture typically includes:

1. Agent definition: Each agent has its own goals, behaviors, and decision-making processes.
2. Communication: Agents communicate with each other to share information and coordinate actions.
3. Environment: The agents interact with their environment, which can be physical or virtual.
4. Control: The system has a control mechanism that manages the interactions between agents a